In [2]:
import open3d as o3d

In [ ]:
from test_data.open3d_example import get_bunny_mesh

In [10]:
import pickle
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import struct

In [11]:
from laserscan import SemLaserScan
import yaml

In [24]:
pc = o3d.io.read_point_cloud("/test_data/eagle.points.ply")
print(pc)
o3d.visualization.draw_geometries([pc])

PointCloud with 796825 points.


In [23]:
np.asarray(pc.colors)

array([[0.98039216, 0.96862745, 0.98039216],
       [0.84705882, 0.83137255, 0.83921569],
       [0.85490196, 0.85882353, 0.84313725],
       ...,
       [0.23137255, 0.25098039, 0.27843137],
       [0.37647059, 0.40784314, 0.39215686],
       [0.34509804, 0.36470588, 0.29411765]])

In [13]:
bunny = get_bunny_mesh()
o3d.visualization.draw_geometries([bunny])


In [ ]:
def save_pc(pcd, filename="pcd.png"):
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False) 
    vis.add_geometry(pcd)
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()
    vis.capture_screen_image(filename, do_render=True)
    vis.destroy_window()

In [ ]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [ ]:
with open("road_list_3d.pkl", "rb") as f:
    data = pickle.load(f)

In [ ]:
data["lidars"]['points_xyz'].shape, data["lidars"]['points_feature'].shape

((178624, 3), (178624, 2))

In [ ]:
data["lidars"]['points_feature'].min(), data["lidars"]['points_feature'].max()

(0.0, 41728.0)

In [ ]:
with open("semantic-kitti-all.yaml", "r") as stream:
    try:
        CFG = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
scan = SemLaserScan(sem_color_dict=CFG['color_map'])
scan.open_scan("000000.bin")
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(scan.points)
pcd.paint_uniform_color([0.6,0.6,0.6])

PointCloud with 125635 points.

In [ ]:
road = o3d.geometry.PointCloud()
road.points = o3d.utility.Vector3dVector(np.array(data))
road.paint_uniform_color([1,0,0])

PointCloud with 6113 points.

In [ ]:
plane_model, inliers = road.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: -0.01x + -0.02y + 1.00z + 1.76 = 0


In [ ]:
o3d.visualization.draw_geometries([pcd, road])

In [ ]:
cl, ind = road.remove_statistical_outlier(20,3)
display_inlier_outlier(road, ind)

Remove statistical outliers: [========================================] 100%
Showing outliers (red) and inliers (gray): 


In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data["lidars"]['points_xyz'])

In [ ]:
pcd = convert_kitti_bin_to_pcd("000000.bin")

In [ ]:
scan = SemLaserScan(sem_color_dict=CFG['color_map'])
scan.open_scan("velodyne/000000.bin")
scan.open_label("velodyne/000000_pred.label")
scan.colorize()

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(scan.points)
pcd.colors = o3d.utility.Vector3dVector(scan.sem_label_color)

In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
bunny = get_bunny_mesh().sample_points_poisson_disk(number_of_points=10000)

In [ ]:
o3d.visualization.draw_geometries([bunny])

In [ ]:
bathtub = o3d.io.read_point_cloud("bathtub_0154.ply")
bathtub.paint_uniform_color([0.6,0.6,.6])

[Open3D WARNING] Read PLY failed: unable to open file: bathtub_0154.ply


RPly: Unable to open file


PointCloud with 0 points.

In [ ]:
o3d.visualization.draw_geometries([bathtub])

In [ ]:
bunny.scale(5, bunny.get_center())

PointCloud with 10000 points.

In [ ]:
bunny.get_center()

array([-0.02666336,  0.09405759,  0.00840688])

In [ ]:
R = bunny.get_rotation_matrix_from_xyz((0.5 * np.pi, 0,0))
bunny = bunny.rotate(R, center=(0,0,0))

In [ ]:
center_z = pcd.get_center()[2]
center_z

-1.0326076569252078

In [ ]:
bunny = bunny.translate(np.array([10,0,-1.76]))

In [ ]:
bunny_in_scene_points = np.concatenate((np.asarray(pcd.points),np.asarray(bunny.points)), axis=0)
bunny_in_scene = o3d.geometry.PointCloud()
bunny_in_scene.points = o3d.utility.Vector3dVector(bunny_in_scene_points)
bunny_in_scene.paint_uniform_color([0.6,.6,.6])
bunny_in_scene.colors[:len(pcd.points)] = o3d.utility.Vector3dVector(np.full((len(pcd.points), 3), 0.6))
bunny_in_scene.colors[len(pcd.points):] = o3d.utility.Vector3dVector(np.full((len(bunny.points), 3), 0))

In [ ]:
o3d.visualization.draw_geometries([bunny_in_scene])

In [ ]:
bunny.get_center() - np.array([2,0,0])

array([ 7.97342844, -0.00832722, -1.11689105])

In [ ]:
_, pt_map = bunny_in_scene.hidden_point_removal(bunny.get_center() - np.array([2,0,0]), 1000)
pcd = pcd.select_by_index(pt_map)
o3d.visualization.draw_geometries([pcd])

In [6]:
wolf = o3d.io.read_point_cloud("./clustering/Wolf_One_ply.ply")

In [8]:
o3d.visualization.draw_geometries([wolf])

In [ ]:
point_cloud= np.loadtxt("./clustering/Wolf_One_ply.ply",skiprows=1)